In [1]:
from trnsprt import model
from pyomo.core.base.DataPortal import DataPortal
import pprint

I had to rewrite the definition of th distance parameter from a notation similar to the GAMS dot-notation to a table notation. Why did this work by initializing the model via the create_instance function but to load the .dat file into the DataPortal?

In [2]:
data = DataPortal()
data.load(filename="trnsprt2.dat")
pp = pprint.PrettyPrinter()
pp.pprint(data._data)

{None: {'capacity_plant': {'san-diego': 600, 'seattle': 350},
        'demand_market': {'chicago': 300, 'new-york': 325, 'topeka': 275},
        'distance_in_thousand_miles': {('san-diego', 'chicago'): 1.8,
                                       ('san-diego', 'new-york'): 2.5,
                                       ('san-diego', 'topeka'): 1.4,
                                       ('seattle', 'chicago'): 1.7,
                                       ('seattle', 'new-york'): 2.5,
                                       ('seattle', 'topeka'): 1.8},
        'markets': {None: ['new-york', 'chicago', 'topeka']},
        'plants': {None: ['seattle', 'san-diego']}}}


Reading in the scalar value didn't work at all. So I had to add it manually to the dict. Why?

In [3]:
data._data[None]["freight_costs_per_case_and_thousand_miles"] = {None: 90}
pp.pprint(data._data)

{None: {'capacity_plant': {'san-diego': 600, 'seattle': 350},
        'demand_market': {'chicago': 300, 'new-york': 325, 'topeka': 275},
        'distance_in_thousand_miles': {('san-diego', 'chicago'): 1.8,
                                       ('san-diego', 'new-york'): 2.5,
                                       ('san-diego', 'topeka'): 1.4,
                                       ('seattle', 'chicago'): 1.7,
                                       ('seattle', 'new-york'): 2.5,
                                       ('seattle', 'topeka'): 1.8},
        'freight_costs_per_case_and_thousand_miles': {None: 90},
        'markets': {None: ['new-york', 'chicago', 'topeka']},
        'plants': {None: ['seattle', 'san-diego']}}}


The \_data dict from the DataPortal can be handed over as the dictionary to initialize the instance

In [4]:
instance = model.create_instance(data=data._data)
instance.pprint()

5 Set Declarations
    distance_in_thousand_miles_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=False, Bounds=None
        Virtual
    freight_costs_per_case_in_thousands_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=False, Bounds=None
        Virtual
    markets : Markets
        Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['chicago', 'new-york', 'topeka']
    plants : Canning plants
        Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['san-diego', 'seattle']
    shipment_quantities_in_cases_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=False, Bounds=None
        Virtual

5 Param Declarations
    capacity_plant : Capacity of a plant in cases
        Size=2, Index=plants, Domain=Any, Default=None, Mutable=False
        Key       : Value
        san-diego :   600
          seattle :   350
    demand_market : Demand of a market in cases
        Size=3, Index=markets, Domain=Any, Default=None, Mutable=False
     

In [5]:
from pyomo.opt import SolverFactory
opt = SolverFactory("glpk")
results = opt.solve(instance)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 153.675
  Upper bound: 153.675
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.020998477935791016
# ----------------------------------------------------------
#   Solution Information
# -------------------------------